In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from pandas_profiling import ProfileReport
df_orig = pd.read_csv('Collisions.csv')
df_orig.shape

(221525, 40)

Now let's have a closer look at what the data looks like. I will use pandas_profiling package to provide details and I will use it to ascertain which columns to drop (identifier columns) and which need some cleaning up

In [2]:

# profile = ProfileReport(df,title='Collisions initial profile')
# profile.to_file('profile.html')

In [3]:
# columns to drop: OBJECTID, INCKEY, COLDETKEY, REPORTNO, LOCATION, STATUS, INJURIES, SERIOUSINJURIES, FATALITIES, SDOTCOLNUM, SEGLANEKEY, CROSSWALKKEY
# columns to keep separate: SEVERITYCODE, SEVERITYDESC, COLLISIONTYPE, SDOT_COLDESC, ST_COLDESC

In [4]:
df = df_orig.copy()
df.drop(columns=['OBJECTID','INCKEY','COLDETKEY','REPORTNO','LOCATION','STATUS','INJURIES','SERIOUSINJURIES','FATALITIES','SDOTCOLNUM','SEGLANEKEY','CROSSWALKKEY','SEVERITYDESC','ST_COLDESC'],inplace=True)

# clean ST_COLCODE
def st_colcode(val):
    try:
        return int(val)
    except:
        return np.nan

df['ST_COLCODE'] = df['ST_COLCODE'].apply(st_colcode)

df.columns

Index([&#39;X&#39;, &#39;Y&#39;, &#39;ADDRTYPE&#39;, &#39;INTKEY&#39;, &#39;EXCEPTRSNCODE&#39;, &#39;EXCEPTRSNDESC&#39;,
       &#39;SEVERITYCODE&#39;, &#39;COLLISIONTYPE&#39;, &#39;PERSONCOUNT&#39;, &#39;PEDCOUNT&#39;,
       &#39;PEDCYLCOUNT&#39;, &#39;VEHCOUNT&#39;, &#39;INCDATE&#39;, &#39;INCDTTM&#39;, &#39;JUNCTIONTYPE&#39;,
       &#39;SDOT_COLCODE&#39;, &#39;SDOT_COLDESC&#39;, &#39;INATTENTIONIND&#39;, &#39;UNDERINFL&#39;,
       &#39;WEATHER&#39;, &#39;ROADCOND&#39;, &#39;LIGHTCOND&#39;, &#39;PEDROWNOTGRNT&#39;, &#39;SPEEDING&#39;,
       &#39;ST_COLCODE&#39;, &#39;HITPARKEDCAR&#39;],
      dtype=&#39;object&#39;)

In [5]:
df['INCDATE'].head(20)

0     2013/03/14 00:00:00+00
1     2006/01/15 00:00:00+00
2     2019/09/09 00:00:00+00
3     2019/12/19 00:00:00+00
4     2013/03/27 00:00:00+00
5     2005/07/07 00:00:00+00
6     2020/07/31 00:00:00+00
7     2013/04/01 00:00:00+00
8     2006/04/11 00:00:00+00
9     2013/04/03 00:00:00+00
10    2013/03/30 00:00:00+00
11    2013/03/31 00:00:00+00
12    2006/06/13 00:00:00+00
13    2019/12/23 00:00:00+00
14    2007/04/17 00:00:00+00
15    2004/09/17 00:00:00+00
16    2019/12/20 00:00:00+00
17    2013/03/27 00:00:00+00
18    2020/05/03 00:00:00+00
19    2019/12/22 00:00:00+00
Name: INCDATE, dtype: object

#### Feature engineering

In [6]:
df['DATE'] = pd.to_datetime(df['INCDATE'],format=r'%Y/%m/%d %H:%M:%S+00')
df['DATE']
df['YEAR'] = df['DATE'].apply(lambda x: x.year)
df['DAYOFYEAR'] = df['DATE'].apply(lambda x: x.dayofyear)


In [7]:
def parse_datetime(val:str):
    if ' ' in val:
        min_char = val.find(' ') + 1
        new_time = time.strptime(val[min_char:],r'%I:%M:%S %p')
        return new_time.tm_hour*60+new_time.tm_min*60
    else:
        return np.nan

df['TIME'] = df['INCDTTM'].apply(parse_datetime)
df['TIME'].replace(to_replace=np.nan, value=int(df['TIME'].mean()),inplace=True)

In [8]:
def combine_date_time(val):
    date = val['DATE']
    hour = int(val['TIME']/3600)
    minute = int((val['TIME']-3600*hour)/60)
    return datetime.datetime.combine(date,datetime.time(hour,minute,0))

df['DATETIME'] = df[['DATE','TIME']].apply(combine_date_time,axis=1)
df['DATETIME'] = df['DATETIME'].apply(lambda x: x.timestamp())    

In [9]:
df.drop(columns=['INCDATE','INCDTTM','DATE','YEAR','TIME','SDOT_COLDESC','EXCEPTRSNDESC','INTKEY','ST_COLCODE'],inplace=True)

In [10]:
df['ADDRTYPE'].replace({'Block':1,'Intersection':2,'Alley':3},inplace=True)
df['EXCEPTRSNCODE'].replace({'NEI':1,'':0,' ':0},inplace=True)

df['INATTENTIONIND'].replace({'Y':1,'':0,' ':0,'1':1,'0':0},inplace=True)
df['INATTENTIONIND'].fillna(0,inplace=True)

df['UNDERINFL'].replace({'Y':1,'N':0},inplace=True)

df['WEATHER'].replace(
    {
        'Clear':0,
        'Overcast':1,
        'Partly Cloudy':1,
        'Raining':2,
        'Snowing':3,
        'Fog/Smog/Smoke':4,
        'Sleet/Hail/Freezing Rain':5,
        'Blowing Sand/Dirt':6,
        'Severe Crosswind':7,
        'Blowing Snow':3,
        'Unknown':np.nan,
        '':np.nan
    },
    inplace=True
)

df['ROADCOND'].replace(
    {
        'Dry':0,
        'Wet':1,
        'Ice':4,
        'Snow/Slush':3,
        'Standing Water':2,
        'Sand/Mud/Dirt':5,
        'Oil':6,
        'Other':7,
        'Unknown':np.nan,
        '':np.nan
    },
    inplace=True
)

df['LIGHTCOND'].replace(
    {
        'Daylight':0,
        'Dusk':1,
        'Dawn':2,
        'Dark - Street Lights On':3,
        'Dark - Street Lights Off':4,
        'Dark - No Street Lights':5,
        'Dark - Unknown Lighting':6,
        'Other':7,
        'Unknown':np.nan,
        '':np.nan
    },
    inplace=True
)

df['PEDROWNOTGRNT'].replace({'Y':1},inplace=True)
df['PEDROWNOTGRNT'].fillna(0,inplace=True)

df['SPEEDING'].replace({'Y':1},inplace=True)
df['SPEEDING'].fillna(0,inplace=True)

df['HITPARKEDCAR'].replace({'N':0,'Y':1},inplace=True)

df['JUNCTIONTYPE'].replace(
    {
        'Unknown':0,
        'Mid-Block (not related to intersection)':1,
        'At Intersection (intersection related)':2,
        'Mid-Block (but intersection related)':3,
        'Driveway Junction':4,
        'At Intersection (but not related to intersection)':5,
        'Ramp Junction':6,
        '':np.nan
    },
    inplace=True
)
df['JUNCTIONTYPE'].fillna(0,inplace=True)
df.fillna(df.mean(),inplace=True)

In [11]:
profile_clean = ProfileReport(df,title='Collisions data profile - cleaned',dark_mode=True)
profile_clean.to_file('profile_clean.html')

Export report to file: 100%|██████████| 1/1 [00:00&lt;00:00, 25.03it/s]


In [13]:
df['LIGHTCOND'].value_counts()

0                          119492
3                           50133
1                            6082
2                            2609
5                            1579
4                            1239
7                             244
Dark - Unknown Lighting        23
Name: LIGHTCOND, dtype: int64